# Evaluation of search query results

### Loading the dataset - documents in which the search needs to be done

In [2]:
cranInp = open('cran.all.1400').read().replace('.T\n','\n').replace('.A\n','\n').replace('.B\n','\n').replace('.W\n','\n').split('\n.I ')

# Figure out what is happening here. You have already seen this.
from sklearn.feature_extraction.text import TfidfVectorizer
Vcount = TfidfVectorizer(analyzer='word', ngram_range=(1,1), stop_words = 'english')
countMatrix = Vcount.fit_transform(cranInp)

### Loading the sample queries

In [3]:
cranQuery = open('cran.qry').read().replace('.W\r','').split('.I ')[1:]

cranQuery[0]
queryDict = dict()
queryVects = dict()

for item in cranQuery:
    stuff = item.split('\r\n\n')
    queryDict[stuff[0]] = stuff[1].strip('\r\n').replace('\r',' ')
    queryVects[stuff[0]] = Vcount.transform([stuff[1].strip('\r\n').replace('\r',' ')])

### Loading the Query relevance Judgements

In [4]:
from collections import defaultdict
queryRel = open('cranqrel').read().split('\n')

queryRelDict = defaultdict(dict)
for item in queryRel:
    stuff = item.split()
    try:
        queryRelDict[stuff[0]][stuff[2]].append(stuff[1])
    except:
        queryRelDict[stuff[0]][stuff[2]] = list()
        queryRelDict[stuff[0]][stuff[2]].append(stuff[1])

In [40]:
# Query 1: "what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft"
from sklearn.metrics.pairwise import cosine_similarity

cosMattf = cosine_similarity(queryVects['001'],countMatrix)
related_docs_indices = cosMattf[0].argsort()[:-11:-1]


for item in related_docs_indices:
    print 'Document', item+1, cosMattf[0][item]

tp = list()
for item in queryRelDict['1'].keys():
    for stuff in related_docs_indices:
        if str(stuff+1)in queryRelDict['1'][item]:
            tp.append(stuff+1)

Document 13 0.330903219959
Document 184 0.287073337305
Document 12 0.22903580906
Document 875 0.19808803912
Document 486 0.197586554438
Document 51 0.182630719804
Document 746 0.177758978159
Document 1268 0.151197563192
Document 327 0.144169282408
Document 792 0.136523347919


### Precision

In [41]:
print tp

#All other entries which are in related_docs_indices but not in queryRelDict['1'] are false positives
precision = 1.0*len(tp)/len(related_docs_indices)

print 'Precision is', precision



[12, 51, 184, 875, 486, 13]
Precision is 0.6


### Recall

In [7]:
recallDocLen = 0
for item in queryRelDict['1'].keys():
    recallDocLen += len(queryRelDict['1'][item])
    
print recallDocLen

recall = 1.0*len(tp)/recallDocLen

print 'recall is', recall

29
recall is 0.206896551724


### Precision and Recall @ K

In [8]:
# change the value for k
k = 30

cosMattf = cosine_similarity(queryVects['001'],countMatrix)
related_docs_indices = cosMattf[0].argsort()[:-1*(k+1):-1]

tp = list()
for item in queryRelDict['1'].keys():
    for stuff in related_docs_indices:
        if str(stuff+1)in queryRelDict['1'][item]:
            tp.append(stuff+1)
            
precision = 1.0*len(tp)/len(related_docs_indices)
recall = 1.0*len(tp)/recallDocLen

print 'Precision is', precision
print 'Recall is', recall



Precision is 0.333333333333
Recall is 0.344827586207


In [9]:
#Average Precision @ Key

relOrNot = [0]*k
for item in queryRelDict['1'].keys():
    for i in range(len(related_docs_indices)):
        if str(related_docs_indices[i]+1) in queryRelDict['1'][item]:
            relOrNot[i] = 1       
            
print relOrNot
avgPs = list()

for i in range(len(relOrNot)):
    if relOrNot[i] == 1:
        print 'P@',i+1,' : ',sum(relOrNot[:i+1])*1.0/(i+1)
        avgPs.append(sum(relOrNot[:i+1])*1.0/(i+1))

        
print 'Average Prcision @ K for query 1 :', sum(avgPs)/len(avgPs)

[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0]
P@ 1  :  1.0
P@ 2  :  1.0
P@ 3  :  1.0
P@ 4  :  1.0
P@ 5  :  1.0
P@ 6  :  1.0
P@ 24  :  0.291666666667
P@ 26  :  0.307692307692
P@ 28  :  0.321428571429
P@ 29  :  0.344827586207
Average Prcision @ K for query 1 : 0.726561513199


### Exercise

MAP - Mean Average Precision is nothing but Average Precision across multiple queries for the document collection. Queries are stored in **queryVects** in vectorial forms. Keys being '001','002','028' etc.

 - With the above code for average precision, find MAP.
 - try to fill **relOrNot** with list comprehension. [Hint](http://stackoverflow.com/questions/35332878/python-list-comprehension-when-item-index-is-required)

In [23]:
DCGrel = [0]*k
for item in queryRelDict['1'].keys():
    for i in range(len(related_docs_indices)):
        if str(related_docs_indices[i]+1) in queryRelDict['1'][item]:
            DCGrel[i] = 5 - int(item)       # In DCG higher value means more relevant. But in the dataset, the relevance is ranked.
                                            # In the Dataset, 5 is not relevant and 1 is most relevant.
            
            


In [36]:
import math

DCG = [int(DCGrel[0])]
summedDCG = [DCG[0]]


for i,item in enumerate(DCGrel[1:]):
    DCG.append(int(item)*1.0/math.log(i+2))
    summedDCG.append(sum(DCG))

In [37]:
for i,item in enumerate(summedDCG[:11]):
    print 'DCG @ Rank ',i+1,' : ',item


DCG @ Rank  1  :  1
DCG @ Rank  2  :  5.32808512267
DCG @ Rank  3  :  7.14856357592
DCG @ Rank  4  :  9.31260613725
DCG @ Rank  5  :  13.0406157446
DCG @ Rank  6  :  14.1568369977
DCG @ Rank  7  :  14.1568369977
DCG @ Rank  8  :  14.1568369977
DCG @ Rank  9  :  14.1568369977
DCG @ Rank  10  :  14.1568369977
DCG @ Rank  11  :  14.1568369977


### Exercise

 - With the above code for DCG, find NDCG. [Hint](https://www.kaggle.com/wiki/NormalizedDiscountedCumulativeGain)
 - Find log to the base of 2. Read documentation for math.log
 - With the code for 'Precision and Recall @ K' section, and from the plotting function introduced in previous notebook, plot the precision-recall curve. (hint - Find PR at each value of k and store them in separate lists to plot.)